In [6]:
# ライブラリのインポート
import time
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import *
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid


# deviceの設定 (cpu or gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



c:\Users\sugay\Anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 指定されたプロシージャが見つかりません。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# data.csvをインポート
df = pd.read_csv('dataframe.csv')
print(df.shape)
print(df.head())
# データのサイズを確認
print(df.shape)
# 型を確認
type(df)

(25000, 101)
     0    1    2    3    4    5    6    7    8    9  ...   91   92   93   94  \
0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0 -1.0   
1 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0 -1.0   
2 -1.0 -1.0 -1.0  1.0  1.0  1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0  1.0  1.0   
3 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...  1.0  1.0  1.0  1.0   
4 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0 -1.0   

    95   96   97   98   99   100  
0 -1.0 -1.0 -1.0 -1.0 -1.0  0.01  
1 -1.0 -1.0 -1.0 -1.0 -1.0  0.01  
2  1.0 -1.0 -1.0 -1.0 -1.0  0.01  
3  1.0  1.0  1.0  1.0  1.0  0.01  
4 -1.0 -1.0 -1.0 -1.0 -1.0  0.01  

[5 rows x 101 columns]
(25000, 101)


pandas.core.frame.DataFrame

In [3]:
# 教師データを分離する
data = df.iloc[:, 0: 100]
teacher_data = df.iloc[:, 100]
print(data)
print(teacher_data)

         0    1    2    3    4    5    6    7    8    9  ...   90   91   92  \
0     -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0   
1     -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0   
2     -1.0 -1.0 -1.0  1.0  1.0  1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0   
3     -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...  1.0  1.0  1.0   
4     -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
24995 -1.0 -1.0 -1.0  1.0 -1.0  1.0 -1.0  1.0 -1.0 -1.0  ...  1.0 -1.0 -1.0   
24996 -1.0 -1.0 -1.0 -1.0 -1.0  1.0  1.0  1.0  1.0  1.0  ...  1.0 -1.0 -1.0   
24997 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0  1.0 -1.0  1.0  ...  1.0 -1.0 -1.0   
24998 -1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0  1.0  1.0 -1.0  ... -1.0 -1.0 -1.0   
24999  1.0  1.0  1.0  1.0  1.0 -1.0 -1.0  1.0 -1.0  1.0  ... -1.0  1.0 -1.0   

        93   94   95   96   97   98   99  
0     -1

In [4]:
# pytorchで機械学習を行うためデータの型を
tensor_df = torch.tensor(df.values, dtype=torch.float32)
# 型を確認
print(type(tensor_df))
# 訓練用データとテスト用データに比率5:5でランダムに分割
X_train, X_test, y_train, y_test = train_test_split(data, teacher_data, train_size=0.5)

<class 'torch.Tensor'>


ここまでで機械学習用のデータを作成することができた．以降，RBMの実装を行っていく

In [ ]:
class RBM(nn.Module):
    """Createing RBM with PyTorch
    Attributes:
        b : bias of visible units
        c : bias of hidden units
        w : weight
        k : Sampling Times
        N_v (int) : number of visible units
        N_h (int) : number of hidden units
        epoch (int) : epoch numbers
        learning_rate : learning rate
        batch_size : batch size 
    """
    def __init__(self, N_v=100, N_h=100, k=15, epoch=5000, learning_rate=0.1, batch_size=100, initial_std=0.01, seed=0, device='cpu'):
        super(RBM, self).__init__()
        self.N_v = N_v
        self.N_h = N_h
        self.device = device
        self.b = torch.zeros(1, N_v, device=device)
        self.b = torch.zeros(1, N_v, device=device)
        self.w = torch.empty((N_h, N_v), device=device).normal_(mean=0, std=initial_std)
        self.k = k
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.batch_size = batch_size

    def visible_to_hidden(self, v):
        """
        Sampling hidden units from visible units
        """
        p = torch.sigmoid(F.linear(v, self.w, self.c))
        return p.bernoulli() 
        
    def hidden_to_visible(self, h):
        """
        Sampling visible units from hidden units
        """
        p = torch.sigmoid(F.linear(h, self.w.t(), self.b))
        return p.bernoulli()
    
    def com_hiddens(self, v):
        """
        Calculating P(h=1|v)
        """
        return torch.sigmoid(F.linear(v, self.w, self.c))

    def sample_v(self, v, gib_num=1):
        """
        Sampling visible units
        """
        v = v.view(-1, self.n_vis)
        v = v.to(self.device)
        h = self.visible_to_hidden(v)
        # Gibbs Sampling 1 ~ k
        for _ in range(gib_num):
            v_gibb = self.hidden_to_visible(h)
            h = self.visible_to_hidden(v_gibb)
        return v, v_gibb

    def sample_ph(self, v):
        """
        Sampling ph
        """
        v = v.view(-1, self.n_vis)
        v = v.to(self.device)
        ph = torch.sigmoid(F.linear(v, self.w, self.c)) 
        h = ph.bernoulli()
        # Gibbs Sampling 1 ~ k
        for _ in range(self.k):
            v_gibb = self.hidden_to_visible(h)
            ph_gibb =  torch.sigmoid(F.linear(v_gibb, self.w, self.c))
            h = ph_gibb.bernoulli()
        return ph, ph_gibb

    def free_energy(self, v):
        """
        Caluculating Free energy
        """
        v_term = torch.matmul(v, self.b.t())
        w_x_h = torch.matmul(v, self.w.t()) + self.c
        h_term = torch.sum(F.softplus(w_x_h), dim=1)
        return -h_term - v_term

    # psuedo likelihoodを計算
    def loss(self, v):
        """
        Caluculating psuedo-likelihood
        """
        flip = torch.randint(0, v.size()[1], (1,))
        v_fliped = v.clone()
        v_fliped[:, flip] = 1 - v_fliped[:, flip]
        free_energy = self.free_energy(v)
        free_energy_fliped = self.free_energy(v_fliped)
        return  v.size()[1]*F.softplus(free_energy_fliped - free_energy)

    def batch_fit(self, v_pos):
        """
        Learning Batch Samples
        """
        # positive part
        ph_pos = self.com_hiddens(v_pos)
        # negative part
        v_neg = self.hidden_to_visible(self.h_samples) 
        ph_neg = self.com_hiddens(v_neg)

        lr = (self.learning_rate) / v_pos.size()[0]
        # Update W
        update = torch.matmul(ph_pos.t(), v_pos) - torch.matmul(ph_neg.t(), v_neg)
        self.w += lr * update
        self.b += lr * torch.sum(v_pos - v_neg, dim=0)
        self.c += lr * torch.sum(ph_pos - ph_neg, dim=0)
        # memory of PCD method
        self.h_samples = ph_neg.bernoulli()


    def fit(self, train_loader, train_tensor):
        self.losses = []
        # initialization of hidden units
        self.h_samples = torch.zeros(self.batch_size, self.n_hid, device=device)
        for epoch in range(self.epoch):
            running_loss = 0.0
            start = time.time()
            for id, (data, target) in enumerate(train_loader):
                data = data.view(-1, self.n_vis)
                data = data.to(self.device)
                target = target.to(self.device)
                self.batch_fit(data)
            # Caluculating Psuedo-likelihood 
            for _, (data, target) in enumerate(train_loader):
                data = data.view(-1, self.n_vis)
                data = data.to(self.device)
                target = target.to(self.device)
                running_loss += self.loss(data).mean().item()
            self.losses.append(running_loss)
            end = time.time()
            print(f'Epoch {epoch+1} pseudo-likelihood = {running_loss:.4f}, {end-start:.2f}s')